Document Retrieval Pipeline
- access to the drive will be granted, if needed, just send uns an email, then you can access all the data files
- we worked a lot with datafiles as the demand of resources was very high and even colab pro crashed frequently 


In [9]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 28.7MB/s 
     |████████████████████████████████| 901kB 40.6MB/s 


In [10]:
from google.colab import drive
import pandas as pd
import re
import math
import nltk
import string
import json
import random
import os

from collections import Counter
from scipy import spatial

#removing stopwords using the nltk library
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

#stemming
from nltk.stem.porter import PorterStemmer

#lemmatization
from nltk.stem import wordnet
nltk.download('wordnet')

#train-test-split
from sklearn.model_selection import train_test_split

#bert/transformers and neural networs
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer, RobertaModel, AutoModel, BertModel
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#unzip the data if you want to create all dictionaries (if they are not saved anywhere)
!unzip ./gdrive/MyDrive/Information_Retrieval_Project/evaluation/wikipedia.zip

In [32]:
df = pd.read_json('./gdrive/MyDrive/Information_Retrieval_Project/qa/wikipedia-train.json')

## Helper Method definition

In [12]:
def create_text_dict():
  """ Returns a dictionary containing key: title; value: doc_text"""
  text_dict = dict()
  text = ""
  for filename in os.listdir("./wikipedia/"):
    if filename.endswith(".txt"):
      with open("./wikipedia/{}".format(filename)) as f:
        text = f.read()
      text_dict[filename.lower()] = text
  return text_dict

def full_text(text):
  """ As we 
      ----------
      text: document text
  """
  text = re.sub("\?", "_", text.lower())
  text = re.sub("\:", "_", text)
  text = re.sub('\"', "_", text)
  text = re.sub('\*', "_", text)

  return text

def create_question_df(train_df, id_title_dict, text_dict):
  """ Creates a dataframe containing questionID, question, list of rel. docs per question
      ----------
      train_df: the wikipedia.train json as dataframe
      id_title_dict: dictionary that maps each doc title to an id
      text_dict: dictionary that contains document titles as keys and texts as values
  """
  questionID = list()
  text_questions = list()
  rel_docs_list = list()

  title_id_dict = {title: id for id, title in id_title_dict.items()}
 
  for ele in train_df['Data']:
    rel_docs_set = list()
    questionID.append(ele['QuestionId'])
    text_questions.append(ele['Question'])
    for rel_doc in ele['EntityPages']:
      title = full_text(rel_doc['Filename'])
      if title_id_dict[title] not in rel_docs_set:
        rel_docs_set.append(title_id_dict[title])
    rel_docs_list.append(rel_docs_set)


  data = {'questionID': questionID,
          'questions': text_questions,
          'rel_docs': rel_docs_list
  }
  return pd.DataFrame(data, columns=data.keys())

def create_question_dict(df):
  """ Creates a dictionary containing key: questionID; value: question 
      ----------
      df: dataframe retrieved from wikipedia_train.json
  """
  question_dict = dict()
  for ele in df['Data']:
    question_dict[ele['QuestionId']] = ele['Question']
  return question_dict

def create_id_title_dict(text_dict):
  """ Creates a dictionary that maps each document title to an integer
      ----------
      text_dict: dictionary with key: title; value: document text
  """
  return {i: title for i, title in enumerate(text_dict.keys())}

def print_dict(any_dict, n=3):
  """ Prints the first n key/value-pairs of any dictionary"""
  i = 0 
  for key, value in any_dict.items():
    print("key: ", key)
    print("value: ", value)
    i += 1
    if i == n:
      break

def load_all_dictionaries():
  """ Loads all saved dictionaries in order to reduce effort and computation time
      of creating each of these dictionaries again
  """
  with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/prep_texts.json') as json_file:
    prep_texts = json.load(json_file)
  with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/prep_questions.json') as json_file:
    prep_questions = json.load(json_file)
  with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/title_fulltext.json') as json_file:
    full_texts = json.load(json_file)
  with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/id_title.json') as json_file:
    id_title = json.load(json_file)
  with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/question_dict.json') as json_file:
    question_dict = json.load(json_file)
  return prep_texts, prep_questions, full_texts, id_title, question_dict

Text Processing and tokenization methods

In [13]:
def tokenize_dict(doc_dict):
  """ Converts the text of a doc_dict to tokens
      ----------
      doc_dict: dictionary with key: title; value: document text
  """
  text_tokens = dict()
  for title in doc_dict.keys():
    text_tokens[title] =  re.findall("[\w']+", doc_dict[title])
  return text_tokens

def tokenize_query(query):
  """ Tokenizes a query
      ----------
      query: question-text
  """
  return re.findall("[\w']+", query)

def get_all_tokens(doc_dict):
  """ returns all tokens within the document dictionary; removes duplicate tokens
      ----------
      doc_dict: dictionary with key: title; value: document text
  """
  tokens = set([token for tokengroup in doc_dict.values() for token in tokengroup])
  return tokens

def get_all_query_tokens(query_dict):
  """ returns the tokens of all queries of the passed query dictionary; removes duplicate tokens
      ----------
      query_dict: dictionary with key: questionID; value: tokenized question
  """
  q_tokens = set()

  for query_rel_tuple in query_dict.values():
    for token in query_rel_tuple[0]:
      q_tokens.add(token)
  return q_tokens

 
def get_query_test_set(query_dict, n=0.7):
  """ samples the query dictionary using fractions train: n; test: 1-n
      ----------
      query_dict: dictionary with key: questionID; value: tokenized question
  """
  s = pd.Series(query_dict)
  training_data , test_data  = [i.to_dict() for i in train_test_split(s, train_size=0.7)]
  return training_data, test_data



In [14]:
def lowercase_text(text):
  """ Lowercases all words within a string
      ----------
      text: string
  """
  return text.lower()

def rem_punct(text):
  """ Removes all punctuation from a string"""

  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

def rem_stopwords(text):
  """ Removes all stopwords from a string"""
  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(text)
  filtered_text = [word for word in word_tokens if word not in stop_words]
  return filtered_text

def s_words(tokenized_text, stemmer): 
  """ Stems all tokens using the passed stemmer
      ----------
      tokenized_text: list of tokens of a text (string)
  """
  stems = [stemmer.stem(word) for word in tokenized_text]
  return stems

def lemmatize_word(tokenized_text, lemma):
  """ lemmatizes all tokens in a list of tokens
      ----------
      tokenized_text: list of tokens of a text (string)
  """
  lemmas = [lemma.lemmatize(word, pos='v') for word in tokenized_text]
  return lemmas

def preprocess_text(text_dict, id_title_dict, norm_type='stemming'):
  """ method that calls stemming/lemmatization for a dictionary containing the texts
      returns a dictionary with key: id; value: list of preprocessed tokens of that text
      ----------
      text_dict: dictionary with key: title; value: document text
      id_title_dict: dictionary which maps each doc_title to an integer
      norm_type: decides if stemming or lemmatization is used
  """
  if norm_type == 'stemming':
    stemmer = PorterStemmer()
    return {i: s_words(rem_stopwords(lowercase_text(rem_punct(text_dict[title]))), stemmer) for i, title in id_title_dict.items()}#, 
  else:
    lemma = wordnet.WordNetLemmatizer() 
    return {i: lemmatize_word(rem_stopwords(lowercase_text(rem_punct(text_dict[title]))), lemma) for i, title in id_title_dict.items()}
    

def preprocess_questions(question_df, norm_type='stemming'):
  """ method that calls stemming/lemmatization for a dictionary containing the texts
      returns a dictionary with key: id; value: tutple(list of question tokens, list of rel doc ids)
      ----------
      question_df: dataframe containing questionID, question, list of rel docs
  """
  if norm_type == 'stemming': 
    stemmer = PorterStemmer()
    return {row[1]['questionID']: (s_words(rem_stopwords(lowercase_text(rem_punct(row[1]['questions']))), stemmer), row[1]['rel_docs']) for row in question_df.iterrows()}
  else:
    lemma = wordnet.WordNetLemmatizer()
    return {row[1]['questionID']: (lemmatize_word(rem_stopwords(lowercase_text(rem_punct(row[1]['questions']))), lemma), row[1]['rel_docs']) for row in question_df.iterrows()}


## Initialization and save *dictionaries*

In [ ]:
#reates: 
#text_dict -> title: full_text
#id_title_dict -> id: title
#question_df -> questionID, question, list of rel docs
#question_dict -> questionID, question
text_dict = create_text_dict()
id_title_dict = create_id_title_dict(text_dict)
question_df = create_question_df(df, id_title_dict, text_dict)
question_dict = create_question_dict(df)

In [ ]:
#create prep_text_dict -> doc_id: tokenized text
prep_text_dict = preprocess_text(text_dict, id_title_dict)

In [ ]:
#create prep_question_dict -> questionID: (tokenized question, list of rel docs)
prep_question_dict = preprocess_questions(question_df)

In [ ]:
import json
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/prep_texts.json', "w") as outfile: 
  json.dump(prep_text_dict, outfile)

In [ ]:
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/prep_questions.json', "w") as outfile: 
  json.dump(prep_question_dict, outfile)

In [ ]:
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/title_fulltext.json', "w") as outfile:
  json.dump(text_dict, outfile)

In [ ]:
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/id_title.json', "w") as outfile:
  json.dump(id_title_dict, outfile)

In [ ]:
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/question_dict.json', "w") as outfile:
  json.dump(question_dict, outfile)

## Load dictionaries

In [ ]:
prep_text_dict, prep_question_dict, text_dict, id_title_dict, question_dict = load_all_dictionaries()

In [ ]:
print(id_title_dict['5240'])
print(text_dict[id_title_dict['5240']][:200])
print("\n")

print(id_title_dict['4605'])
print(text_dict[id_title_dict['4605']][:200])
print("\n")

print(id_title_dict['61783'])
print(text_dict[id_title_dict['61783']][:200])
print("\n")

print(id_title_dict['578'])
print(text_dict[id_title_dict['578']][:200])
print("\n")

print(id_title_dict['10361'])
print(text_dict[id_title_dict['10361']][:200])
print("\n")

print(id_title_dict['70942'])
print(text_dict[id_title_dict['70942']][:200])



i = 0
for k, v in prep_question_dict.items():
  if i < 3:
    print(k, v)
    i+=1
  if i == 3:
    if k == 'qw_1376':
      print(k, v)

england.txt
England is a country that is part of the United Kingdom.   It shares land borders with Scotland to the north and Wales to the west. The Irish Sea lies northwest of England and the Celtic Sea lies to t


judi_dench.txt
Dame Judith Olivia "Judi" Dench,  (born 9 December 1934)  is an English actress and author.  Dench made her professional debut in 1957 with the Old Vic Company. Over the following few years she perfor


nation_state.txt
A nation state is a type of state that conjoins the political entity of a state to the cultural entity of a nation, from which it aims to derive its political legitimacy to rule and potentially its st


angola.txt
Angola, officially the Republic of Angola (; Kikongo, Kimbundu and Umbundu: Repubilika ya Ngola), is a country in Southern Africa. It is the seventh-largest country in Africa, and is bordered by Namib


angolan_civil_war.txt
The Angolan Civil War () was a major civil conflict in Angola, beginning in 1975 and continuing, with some inte

## compute inverted index

### Create new train/test-split

In [ ]:
#split up the data into a train and test dictionary; also remove duplicate query tokens
prep_question_train, prep_question_test =get_query_test_set(prep_question_dict, n=0.7)
query_train_tokens = get_all_query_tokens(prep_question_train)


original lenght:  61888
test data length:  18567


In [ ]:
#save the train_dict
with open('./gdrive/MyDrive/Information_Retrieval_Project/train_prep_question_70.json', "w") as outfile:
  json.dump(prep_question_train, outfile)

In [ ]:
#save the test_dict
with open('./gdrive/MyDrive/Information_Retrieval_Project/test_prep_question_30.json', "w") as outfile:
  json.dump(prep_question_test, outfile)

In [ ]:
#load if usage of test set is wanted
query_test_tokens = get_all_query_tokens(prep_question_test)

In [ ]:
#load if usage of train set is wanted
query_test_tokens = get_all_query_tokens(prep_question_train)

In [ ]:
print(len(query_test_tokens))
print(len(query_train_tokens))

### Load train/test-dict

In [ ]:
#load the train_dict
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/train_prep_question_70.json') as json_file:
  prep_question_train = json.load(json_file)

In [74]:
#load the test_dict
with open('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/test_prep_question_30.json') as json_file:
  prep_question_test = json.load(json_file)

In [ ]:
#create set of query tokens form train set
query_train_tokens = get_all_query_tokens(prep_question_train)

In [ ]:
#create set of query tokens form test set
query_test_tokens = get_all_query_tokens(prep_question_test)

In [ ]:
print(len(query_test_tokens))
print(len(query_train_tokens))

21289
33375


### Initialize inverted index

In [ ]:
def compute_inverted_index(query_tokens, prep_text_dict):
  """ computes an inverted index that maps query_id to the documents it appears in
      also counts num of appearences
      ----------
      query_tokens: set of tokens in the query
      prep_text_dict: dictionary -> text_id : tokenized text
  """
  inverted_index = dict()
  for q_token in query_tokens:
    inverted_index[q_token] = dict()
  for doc, doc_text in prep_text_dict.items():
    for token in doc_text:
      if token in inverted_index.keys():
        if doc in inverted_index[token].keys():
          inverted_index[token][doc] += 1
        else:
          inverted_index[token][doc] = 1
  return inverted_index

def show_inv_n(inverted_index, n=3):
  k = 0
  for token, occurences in inverted_index.items():
    print("token: ", token)
    print("occurences: ", occurences)
    k += 1
    if k == n:
      break

In [ ]:
#create an inverted index for the train set
inv_train = compute_inverted_index(query_train_tokens, prep_text_dict)
show_inv_n(inv_train)

In [ ]:
#create an inverted index for the test set
inv_test = compute_inverted_index(query_test_tokens, prep_text_dict)
show_inv_n(inv_test)

token:  lablach
occurences:  {'16779': 1, '18208': 1, '20192': 5, '23723': 1, '36632': 1, '40635': 2, '41405': 1, '63608': 1, '69749': 1}
token:  tenburi
occurences:  {'4330': 1, '10486': 2, '16022': 2, '20700': 1, '21923': 1, '23031': 10, '31634': 5, '35154': 2, '47531': 1, '48618': 2, '49974': 1, '58729': 2, '63377': 1, '63420': 1, '68720': 26}
token:  carrier
occurences:  {'30': 1, '40': 1, '56': 1, '117': 1, '137': 1, '142': 4, '185': 1, '208': 1, '239': 1, '257': 1, '280': 1, '417': 10, '418': 1, '430': 190, '441': 1, '536': 1, '539': 1, '560': 1, '611': 3, '674': 4, '686': 1, '735': 1, '750': 18, '764': 1, '774': 1, '814': 1, '902': 2, '919': 1, '921': 2, '943': 1, '954': 1, '1002': 3, '1032': 2, '1037': 1, '1164': 11, '1172': 3, '1199': 1, '1200': 1, '1243': 1, '1245': 1, '1300': 1, '1343': 1, '1380': 1, '1392': 1, '1433': 1, '1469': 3, '1501': 1, '1504': 1, '1545': 15, '1587': 1, '1593': 1, '1612': 2, '1668': 1, '1678': 1, '1703': 1, '1735': 1, '1746': 31, '1751': 8, '1835': 1,

## BM25 and Variants

In [ ]:
from itertools import islice
import gc


#implemeetn the BIM without relevance judgement
def compute_BIM_scores(inverted_index, text_files, prob_type=''):
  """ Method that is called by the compute_BIM method below
      ----------
      inverted_index: inverted index containing all the question tokens as keys
      and the corresponding list of documents they appear in; also contains frequency of occurences in each doc
      text_files: dictionary with key: doc_id; value: preprocessed text tokens
      prob_type: the method which is used for relevance score computation
  """
  final_scores = dict()
  total_num_docs = len(text_files)
  total_doc_length = 0
  for i, doc in enumerate(text_files.values()):
    total_doc_length += len(doc)
  avg_doc_length = total_doc_length / total_num_docs

  for token in inverted_index.keys():
    for doc in inverted_index[token]:
      if doc not in final_scores.keys():
        final_scores[doc] = 0
      wt = math.log(0.5 * (total_num_docs/len(inverted_index[token])))

      if prob_type == 'two_poisson':
        final = wt * ((inverted_index[token][doc]*(2.5))/(inverted_index[token][doc]+1.5))
      elif prob_type == 'BM11':
        doc_length = len(text_files[doc])
        final = wt * ((inverted_index[token][doc]*(2.5))/(inverted_index[token][doc]+1.5 * (doc_length / avg_doc_length)))
      elif prob_type == 'BM25':
        b = 0.5
        doc_length = len(text_files[doc])
        final = wt * ((inverted_index[token][doc]*(2.5))/(inverted_index[token][doc]+1.5 * (doc_length / avg_doc_length) * b + 1.5*(1-0.75)))
      elif prob_type == 'BM25+':
        b = 0.5
        delta = 1
        doc_length = len(text_files[doc])
        final = wt * ((inverted_index[token][doc]*(2.5))/(inverted_index[token][doc]+1.5 * (doc_length / avg_doc_length) * b + 1.5*(1-0.75))+delta)
      else:
        final = wt
      final_scores[doc] += final
  return final_scores

def compute_BIM(prep_question_dict, inverted_index, method='BM25+'):
  """ Main BM-Method; Uses the prerocessed question dictionary and the inverted index
      containing to that question tokens to compute the scores and return a dataframe
      which contains the ranked documents based on that computed relevance score;
      Each result dataframe is stored in a list, which is then, for RAM-reasons,
      saved in google drive as a concatenated dataframe after each 10000 questions. 
      ----------
      prep_question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
      inverted index: inverted index of the query tokens considering all document
      method: type of relevance score
  """
  final_dfs = list()
  i = 0
  for questionID, rel_tuple in prep_question_dict.items():
    question_inverted_index = dict()
    for token in rel_tuple[0]:
      question_inverted_index[token] = inverted_index[token]
    final_results = compute_BIM_scores(question_inverted_index, prep_text_dict, prob_type=method)
    ordered = {k: v for k, v in sorted(final_results.items(), key=lambda item: item[1], reverse=True)}
    df = pd.DataFrame(ordered.items(), columns=['doc', 'score'])[:50]
    df.insert(0,'question',questionID)
    final_dfs.append(df) 
    if i % 500 == 1:
      print(i)
      display(final_dfs[0].head())
      gc.collect()
    if (i%10000 == 5000):
      final_df = pd.concat(final_dfs)
      final_df.to_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm11_test_{}.csv'.format(i), sep='\t')
      del final_df
      final_dfs = list()
    i += 1
  final_df = pd.concat(final_dfs)
  final_df.to_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm11_test_{}.csv'.format("last"), sep='\t')

compute_BIM(prep_question_test, inv_test, method='BM11')


1


,question,doc,score
0,bt_2498,38272,36.827641
1,bt_2498,37863,33.743715
2,bt_2498,66197,18.765094
3,bt_2498,9159,18.676479
4,bt_2498,54312,16.850490


501


,question,doc,score
0,bt_2498,38272,36.827641
1,bt_2498,37863,33.743715
2,bt_2498,66197,18.765094
3,bt_2498,9159,18.676479
4,bt_2498,54312,16.850490


1001


...

,question,doc,score
0,sfq_23151,62422,13.850858
1,sfq_23151,47455,12.881140
2,sfq_23151,1728,12.524259
3,sfq_23151,57834,12.521499
4,sfq_23151,19585,12.485555


18001


,question,doc,score
0,sfq_23151,62422,13.850858
1,sfq_23151,47455,12.881140
2,sfq_23151,1728,12.524259
3,sfq_23151,57834,12.521499
4,sfq_23151,19585,12.485555


18501


,question,doc,score
0,sfq_23151,62422,13.850858
1,sfq_23151,47455,12.881140
2,sfq_23151,1728,12.524259
3,sfq_23151,57834,12.521499
4,sfq_23151,19585,12.485555


In [ ]:
#compute BIM for train
compute_BIM(prep_question_train, inv_train)

In [ ]:
#compute BIM for test
compute_BIM(prep_question_test, inv_test)

# Evaluation BM25

## Precision Method definition

In [15]:
#methods used for evaluation

def assign_labels(row, question_dict):
  """ Returns 1 if doc id in the row is in the list of relevant documents of the 
      question_dict (doc is actually relevant) and 0 if not
      ----------
      results_df_list: list of result dataframes
      question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
  """
  if int(row['doc']) in question_dict[row['question']][1]:
    return 1
  else:
    return 0

def add_labels(results_df_list, question_dict):
  """ Adds a new column to all result dataframes in the list. Column contains 1
      if document is actually relevant to a question and 0 if not.
      ----------
      results_df_list: list of result dataframes
      question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
  """
  for result_df in results_df_list:
    result_df['label'] = result_df.apply(lambda row: assign_labels(row, question_dict), axis=1)

def fraction_found(results_df_list, question_dict):
  found = 0 
  for result_df in results_df_list:
    found += result_df['label'].sum()
  
  num_relevant = 0
  for q_rel_tuple in questions_dict.values():
    num_relevant += sum(q_rel_tuple[1])

  return found, num_relevant, found / num_relevant

def PK(result_df, k=5):   
  """ Computes the precision @k for a single query
      ----------
      result_df: dataframe containing scores for document considering one query
  """
  return result_df.iloc[0:k]['label'].sum() / k

def APK(result_df, k=5):
  """ Computes the Average precision @k for a single query
      ----------
      result_df: dataframe containing scores for document considering one query
  """
  apk = 0
  count = 0 
  for i, row in enumerate(result_df.iterrows()):
    if row[1]['label'] == 1:
      count += 1
      apk += result_df.iloc[0:i+1]['label'].sum() / (i+1)
    # just remove this to get AP without k
    if i == k-1:
      break
  if count == 0:
    return count
  return apk / count

def RPrec(result_df, question_dict):
  """ Computes the R-Precision for a single query
      ----------
      result_df: dataframe containing scores for document considering one query
      question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
  """
  return PK(result_df, len(question_dict[result_df.iloc[1]['question']][1]))

def ARPrec(result_df, question_dict):
  """ Computes the Average R-Precision for a single query
      ----------
      result_df: dataframe containing scores for document considering one query
      question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
  """
  return APK(result_df, len(question_dict[result_df.iloc[1]['question']][1]))

def meanPrecisions(result_df_list, question_dict, k=5, PType='P@K'):
  """ Computes the Mean of the different precision types considering PType
      ----------
      result_dfllist: list of dataframes containing scores for document considering one query
      question_dict: dict with key: question_id; value: tuple(question tokens, rel doc list)
      k: parameter k for P@K and AP@K
      PType: type of precision to compute
  """
  mean = 0
  for df in result_df_list:
    if PType == 'P@K':
      mean += PK(df, k)
    elif PType == 'AP@K':
      mean += APK(df, k)
    elif PType == 'R-Precision':
      mean += RPrec(df, question_dict)
    elif PType == 'AR-Precision':
      mean += ARPrec(df, question_dict)
    else:
      print("No valid evaluation matrix chosen; use one of P@K', 'AP@K', 'R-Precision' or 'AR-Precision' as input values for the PType")
      break
  return mean/len(result_df_list)


def add_missed_docs(result_df_list, prep_question_dict): 
  """ this method takes a list of result dataframes with the corresponding labels as input
      each dataframe gets converted into a dataframe where relevant and non-relevant docs are equally distributed
      ----------
      result_dfllist: list of dataframes containing scores for document considering one query
      prep_question_dict: dict with key: question_id; value: tokenized question
  """
  results_df_list = list()
  for i, result_df in enumerate(result_df_list):
    rel_docs_list = prep_question_dict[result_df.iloc[0]['question']][1]
    result_df = pd.concat([result_df.loc[result_df['label'] == 0][0:len(rel_docs_list)], result_df.loc[result_df['label'] == 1]])   
    if result_df['label'].sum() < len(rel_docs_list):
      for doc in rel_docs_list:
        if doc not in list(result_df['doc']):
          result_df.loc[len(result_df)] = [result_df.iloc[0]['question']] + [doc] + [0] + [1]
    result_df.reset_index()
    results_df_list.append(result_df)
  return results_df_list

def create_train_df(result_df_list, prep_question_dict):
  """ creates train dataframe as described in the report using the add_missed_docs function
      ----------
      result_dfllist: list of dataframes containing scores for document considering one query
      prep_question_dict: dict with key: question_id; value: tokenized question
  """
  results_df_list = add_missed_docs(result_df_list, prep_question_dict)
  train_df = pd.concat(results_df_list)
  train_df = train_df.reset_index(drop=True)
  return train_df

def add_full_question(row, question_dict):
  """ adds a full question text a dataframe using the questin_dict
      ----------
      question_dict: question_id -> full question
  """
  return question_dict[row['question']]

def add_full_doc(row, text_dict, id_doc_dict):
  """ adds a full doc text a dataframe using the questin_dict
      ----------
      text_dict: title -> full text
      id_doc_dict: id -> title
  """
  return text_dict[id_doc_dict[str(row['doc'])]]

def add_full_question_texts(train_df, question_dict):
  """ adds a full question text a dataframe using the questin_dict"""

  train_df['full_question'] = train_df.apply(lambda row: add_full_question(row, question_dict), axis=1)

def add_full_doc_texts(train_df, question_dict, id_doc_dict):
  """ adds a full doc text a dataframe using the questin_dict"""

  train_df['full_doc'] = train_df.apply(lambda row: add_full_doc(row, text_dict, id_doc_dict), axis=1)

## Loading the test dataset result frames



*   For compuatational reasons we use the test-split (30% of the questions) in order to compute the different precision scores and compare the models




In [ ]:
#load the result dataframes of the testset bm25
df_list = list()
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_test_5000.csv', sep='\t'))
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_test_15000.csv', sep='\t'))
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_test_last.csv', sep='\t'))

In [6]:
#load the result dataframes of the testset bm25+
df_list2 = list()
df_list2.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_5000.csv', sep='\t'))
df_list2.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_15000.csv', sep='\t'))
df_list2.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_last.csv', sep='\t'))

In [ ]:
#load results of bm11 
df_list3 = list()
df_list3.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm11_test_5000.csv', sep='\t'))
df_list3.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm11_test_15000.csv', sep='\t'))
df_list3.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm11_test_last.csv', sep='\t'))

In [ ]:
#concate the dataframes into one large dataframe
df_results = pd.concat(df_list)
del df_results['Unnamed: 0']
display(df_results.head(30))

,question,doc,score
0,bt_2498,38272,52.852304
1,bt_2498,37863,48.663329
2,bt_2498,66197,40.346682
3,bt_2498,51957,26.050616
4,bt_2498,9159,25.920588
5,bt_2498,1767,25.446150
6,bt_2498,37204,24.281814
7,bt_2498,53659,23.736476
8,bt_2498,21554,23.016751
9,bt_2498,22446,22.802304


In [ ]:
df_results2 = pd.concat(df_list2)
del df_results2['Unnamed: 0']
display(df_results2.head(30))

,question,doc,score
0,bt_2498,38272,34.686048
1,bt_2498,37863,30.497073
2,bt_2498,66197,22.180426
3,bt_2498,9159,17.600086
4,bt_2498,51957,16.204863
5,bt_2498,37204,14.693858
6,bt_2498,6532,14.544418
7,bt_2498,54312,13.939712
8,bt_2498,4180,13.725870
9,bt_2498,22446,13.701241


In [ ]:
df_results3 = pd.concat(df_list3)
del df_results3['Unnamed: 0']
display(df_results3.head(30))

,question,doc,score
0,bt_2498,38272,36.827641
1,bt_2498,37863,33.743715
2,bt_2498,66197,18.765094
3,bt_2498,9159,18.676479
4,bt_2498,54312,16.850490
5,bt_2498,4180,16.238850
6,bt_2498,9566,15.967551
7,bt_2498,40853,14.571041
8,bt_2498,6532,14.454412
9,bt_2498,22446,14.389599


In [ ]:
#split the dataframes according to the question
df_result_list = list()
for question, df_question in df_results.groupby('question'):
    df_result_list.append(df_question)

In [ ]:
#split the dataframes according to the question
df_result_list2 = list()
for question, df_question in df_results2.groupby('question'):
    df_result_list2.append(df_question)

In [ ]:
#split the dataframes according to the question
df_result_list3 = list()
for question, df_question in df_results3.groupby('question'):
    df_result_list3.append(df_question)

In [ ]:
#add a label columnn to the result dataframes in order to evaluate them
add_labels(df_result_list, prep_question_dict)

NameError: ignored

In [ ]:
#add a label columnn to the result dataframes in order to evaluate them
add_labels(df_result_list2, prep_question_dict)

In [ ]:
#add a label columnn to the result dataframes in order to evaluate them
add_labels(df_result_list3, prep_question_dict)

In [ ]:
#print some result dataframes
#for reranking: 50, 52, 8
#for here: 1002,104
display(df_result_list[104])
display(df_result_list[1002])

In [ ]:
#compute P@K, AP@K, R-Precision and Average R-Precision based on one dataframe for different values of K
for i in range(1,8):
  print(i)
  print("P@K: ",PK(df_result_list[1002],i))
  print("AP@K: ", APK(df_result_list[1002],i))
print()
print("RPrec: ",RPrec(df_result_list[1002], prep_question_test))
print("ARPrec: ", ARPrec(df_result_list[1002], prep_question_test))

1
P@K:  1.0
AP@K:  1.0
2
P@K:  0.5
AP@K:  1.0
3
P@K:  0.3333333333333333
AP@K:  1.0
4
P@K:  0.5
AP@K:  0.75
5
P@K:  0.4
AP@K:  0.75
6
P@K:  0.3333333333333333
AP@K:  0.75
7
P@K:  0.2857142857142857
AP@K:  0.75

RPrec:  0.5
ARPrec:  1.0


In [ ]:
#compute mean scores for the list of the two inspected dataframes
test_list = [df_result_list[1002], df_result_list[104]]
for i in range(1, 8):
  MPK = meanPrecisions(df_result_list, prep_question_test, k=i, PType='P@K')
  print("MPK: k=",i,": ",MPK)
  MAPK = meanPrecisions(df_result_list, prep_question_test, k=i, PType='AP@K')
  print("MAPK: k={}: ".format(i),MAPK)
MRPrecision = meanPrecisions(df_result_list, prep_question_test, k=5, PType='R-Precision')
print("MRP: ", MRPrecision)
MARPrecision = meanPrecisions(df_result_list, prep_question_test, k=5, PType='AR-Precision')
print("MARP: ", MARPrecision)


MPK: k= 1 :  0.4898475790380783
MAPK: k=1:  0.4898475790380783
MPK: k= 2 :  0.36010125491463346
MAPK: k=2:  0.5599450638229116
MPK: k= 3 :  0.28451912892047587
MAPK: k=3:  0.5769689951706483
MPK: k= 4 :  0.23465018581354016
MAPK: k=4:  0.5817744145826274
MPK: k= 5 :  0.2001400333925513
MAPK: k=5:  0.5828943825066073
MPK: k= 6 :  0.17462882174465755
MAPK: k=6:  0.5821485102003033
MPK: k= 7 :  0.15512160592143098
MAPK: k=7:  0.5811305003500871
MRP:  0.4182936788769691
MARP:  0.5290221804112454


In [ ]:
#compute mean scores for the list of the two inspected dataframes
test_list = [df_result_list[1002], df_result_list[104]]
for i in range(1, 8):
  MPK = meanPrecisions(df_result_list2, prep_question_test, k=i, PType='P@K')
  print("MPK: k=",i,": ",MPK)
  MAPK = meanPrecisions(df_result_list2, prep_question_test, k=i, PType='AP@K')
  print("MAPK: k={}: ".format(i),MAPK)
MRPrecision = meanPrecisions(df_result_list2, prep_question_test, k=5, PType='R-Precision')
print("MRP: ", MRPrecision)
MARPrecision = meanPrecisions(df_result_list2, prep_question_test, k=5, PType='AR-Precision')
print("MARP: ", MARPrecision)

MPK: k= 1 :  0.5194700274680886
MAPK: k=1:  0.5194700274680886
MPK: k= 2 :  0.3790326924112673
MAPK: k=2:  0.5906985511929768
MPK: k= 3 :  0.297463241234433
MAPK: k=3:  0.6061336421249255
MPK: k= 4 :  0.24527387300048473
MAPK: k=4:  0.6107774845454673
MPK: k= 5 :  0.2082188829643704
MAPK: k=5:  0.6107121805114327
MPK: k= 6 :  0.1810829249025886
MAPK: k=6:  0.6102258786497039
MPK: k= 7 :  0.16060752948782298
MAPK: k=7:  0.6091144084786081
MRP:  0.44262430328030317
MARP:  0.5575267837867328


In [ ]:
#compute mean scores for the list of the two inspected dataframes
test_list = [df_result_list[1002], df_result_list[104]]
for i in range(1, 8):
  MPK = meanPrecisions(df_result_list3, prep_question_test, k=i, PType='P@K')
  print("MPK: k=",i,": ",MPK)
  MAPK = meanPrecisions(df_result_list3, prep_question_test, k=i, PType='AP@K')
  print("MAPK: k={}: ".format(i),MAPK)
MRPrecision = meanPrecisions(df_result_list3, prep_question_test, k=5, PType='R-Precision')
print("MRP: ", MRPrecision)
MARPrecision = meanPrecisions(df_result_list3, prep_question_test, k=5, PType='AR-Precision')
print("MARP: ", MARPrecision)

MPK: k= 1 :  0.46728065923412504
MAPK: k=1:  0.46728065923412504
MPK: k= 2 :  0.34453600473959173
MAPK: k=2:  0.5385899714547315
MPK: k= 3 :  0.2757042063876706
MAPK: k=3:  0.5576156621963738
MPK: k= 4 :  0.2291565681047019
MAPK: k=4:  0.5633307002268102
MPK: k= 5 :  0.1963052727957942
MAPK: k=5:  0.565394322663275
MPK: k= 6 :  0.17184610689214172
MAPK: k=6:  0.564869122636943
MPK: k= 7 :  0.15281336318662406
MAPK: k=7:  0.5643499005318388
MRP:  0.39992508598502824
MARP:  0.5082911554366562


In [ ]:
#compute mean scores for the whole test set
MPK = meanPrecisions(df_result_list, prep_question_test, k=5, PType='P@K')
MAPK = meanPrecisions(df_result_list, prep_question_test, k=5, PType='AP@K')
MRPrecision = meanPrecisions(df_result_list, prep_question_test, k=5, PType='R-Precision')
MARPrecision = meanPrecisions(df_result_list, prep_question_test, k=5, PType='AR-Precision')

print(MPK)
print(MAPK)
print(MRPrecision)
print(MARPrecision)

0.2001400333925513
0.5828943825066073
0.4182936788769691
0.5290221804112454


# monoBERT

## Dataset/Loader and Model definition

In [16]:
# 1 - Dataset for monoBERT is prepared
# monoBERT_dataset class and its properties are defined
class monoBERT_dataset(Dataset):
  def __init__(self):
    csv_path = "./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_train.csv"
    self.dataset = pd.read_csv(csv_path, sep='\t')

  def __len__(self): 
    return len(self.dataset)
  
  def __getitem__(self, itemID):
    row = self.dataset.iloc[itemID]
    return row['full_question'], row['full_doc'], row['label']

# 2 - Dataloader is prepared to iterate over batches 
# Get a dataloader for monoBERT
# Batch size = 16 (max amount the gpu can handle); shuffle = true
def get_dataloader_monoBERT():
  monoBERT_data = monoBERT_dataset()
  dataloader = DataLoader(dataset=monoBERT_data, batch_size=16, shuffle=True, num_workers=1)
  return dataloader

In [17]:
# weights initialization for the linear model using xavier initialization function
# more stable and faster training
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.zero_(m.bias)

#monobert model with pretrained distilroberta-base and a linear model for the output
#takes the [CLS]-embedding of the BERT model for as input of the linear model
class monoBERT(nn.Module):
  def __init__(self):
    super(monoBERT, self).__init__()
    self.bert = AutoModel.from_pretrained('distilroberta-base')
    self.decision = nn.Sequential(
            nn.Linear(768, 128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,1)
        )
    weights_init(self.decision)
  def forward(self, ids, mask):
    outputs = self.bert(
               ids, 
               attention_mask=mask)
    
    sequence_output = outputs.last_hidden_state
    x = sequence_output[:,0,:].contiguous().view(-1,768)

    decision_output = self.decision(x) ## extract the 1st token's embeddings
    return decision_output

In [18]:
# Tokenizer for monoBERT
def tokenize_monoBERT(question, text, tokenizer):
  # creates: [CLS] question [SEP] Doc [SEP], max 512 tokens
  tokenized_input = tokenizer(question, text, return_tensors='pt', max_length=512, padding=True, truncation='only_second')
  return tokenized_input

In [19]:
#Get a monoBERT model and tokenizer
def get_monoBERT_and_tokenizer():
  return monoBERT(), AutoTokenizer.from_pretrained('distilroberta-base', use_fast=True)

In [20]:
def load_monoBERT(path):
  model = monoBERT()
  model.load_state_dict(torch.load(path)["model_state_dict"])
  model.eval()
  return model

## Training Preparation



*   Execute these cells if you want to train the model
*   First the results of bm25 on the training set are loaded 
*   Then the dataset is prepared by first adding labels to the result dataframes and then creating a dataframe which contains all relevant documents from the training json and the same number of irrelevant documents for each query
* These irrelevant documents are the top n irrelavant documents from the bm25  output dataframes, to make the training as specific and difficult for monoBERT as possible



In [ ]:
df_list = list()
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_train_10000.csv', sep='\t'))
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_train_22000.csv', sep='\t'))
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_train_34000.csv', sep='\t'))
df_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25_train_last.csv', sep='\t'))

In [ ]:
#concate the dataframes into one large dataframe
df_train = pd.concat(df_list)
del df_train['Unnamed: 0']
display(df_train.head(30))

,question,doc,score
0,sfq_24649,19446,35.506735
1,sfq_24649,73063,26.827606
2,sfq_24649,11848,25.706211
3,sfq_24649,45493,24.775058
4,sfq_24649,62422,24.552775
5,sfq_24649,17754,23.573254
6,sfq_24649,65288,22.296650
7,sfq_24649,26802,22.122978
8,sfq_24649,64876,21.665808
9,sfq_24649,42152,21.625962


In [ ]:
#split the dataframes according to the question
df_train_list = list()
for question, df_question in df_train.groupby('question'):
    df_train_list.append(df_question)

In [ ]:
#add a label columnn to the result dataframes in order to evaluate them
add_labels(df_train_list, prep_question_dict)

In [ ]:
train_df = create_train_df(df_train_list, prep_question_dict)

In [ ]:
display(train_df.head(20))
print(len(train_df))

,question,doc,score,label
0,bb_10,27062,26.281998,0
1,bb_10,72724,31.566161,1
2,bb_1000,71300,51.199494,0
3,bb_1000,5640,46.079267,0
4,bb_1000,72730,41.535001,1
5,bb_1000,11920,37.791938,1
6,bb_1011,37131,71.760624,0
7,bb_1011,62787,86.785895,1
8,bb_1014,11682,58.728495,0
9,bb_1014,16945,92.952497,1


154535


In [ ]:
#remove the bm25-scoring
del train_df['score']
display(train_df.head())

,question,doc,label
0,bb_10,27062,0
1,bb_10,72724,1
2,bb_1000,71300,0
3,bb_1000,5640,0
4,bb_1000,72730,1


In [ ]:
#add full_questions to the dataframe
add_full_question_texts(train_df, question_dict)
display(train_df.head())

,question,doc,label,full_question
0,bb_10,27062,0,In the US TV comedy show Everybody Loves Raymo...
1,bb_10,72724,1,In the US TV comedy show Everybody Loves Raymo...
2,bb_1000,71300,0,An adaptation of which classic 1905 story by E...
3,bb_1000,5640,0,An adaptation of which classic 1905 story by E...
4,bb_1000,72730,1,An adaptation of which classic 1905 story by E...


In [ ]:
#add full_texts to the dataframe
add_full_doc_texts(train_df, text_dict, id_title_dict)
display(train_df.head())

,question,doc,label,full_question,full_doc
0,bb_10,27062,0,In the US TV comedy show Everybody Loves Raymo...,Becker is an American sitcom that ran from 199...
1,bb_10,72724,1,In the US TV comedy show Everybody Loves Raymo...,Everybody Loves Raymond is an American televis...
2,bb_1000,71300,0,An adaptation of which classic 1905 story by E...,The Railway Children is a children's book by E...
3,bb_1000,5640,0,An adaptation of which classic 1905 story by E...,John Barrymore (born John Sidney Blyth; Februa...
4,bb_1000,72730,1,An adaptation of which classic 1905 story by E...,Edith Nesbit (married name Edith Bland; 15 Aug...


In [ ]:
train_df.to_csv('./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_train.csv', sep='\t')


## Model initilization and training

In [ ]:
#initialize the dataloader
loader = get_dataloader_monoBERT()

In [ ]:
#or initialize from checkpoint
model = load_monoBERT('./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_lr_0.0001_DROPOUT_Adam_epoch_3.tar')

In [ ]:
#initialize model and tokenizer; work on GPU (cuda)
_, tokenizer = get_monoBERT_and_tokenizer()


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializin

In [ ]:
model = monoBERTCNN()
model.to('cuda')
print()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Criterion and Optimizer are defined for training
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [ ]:
#train the model
epochs = 100

for j, epoch in enumerate(range(epochs)):
  avg_loss = 0 
  print("EPOCH")
  for i, batch in enumerate(loader):
    question = batch[0]
    paragraph = batch[1] ## assuming that data loader returns a tuple of data and its targets
    target = batch[2].to('cuda')


    optimizer.zero_grad()   
    encoding = tokenize_monoBERT(question, paragraph, tokenizer)

    #get token_ids and attention_mask from encoding
    input_ids = encoding['input_ids'].to('cuda')
    attention_mask = encoding['attention_mask'].to('cuda')
    outputs = model(input_ids, attention_mask)
    outputs = outputs.flatten()
    loss = criterion(outputs, target.to(torch.float32))
    avg_loss += loss

    if i % 10 == 5:
      print("Batch_loss: ", loss)
      print("Item:{} - avg_loss: {}".format(i*16,avg_loss/i))

    if i == 5000:
      print("Save the weights")
      torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
      }, "./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_lr_0.0001_DROPOUT_Adam_epoch_{}.tar".format(epoch))
    
    loss.backward()
    optimizer.step()

## Prediction Making

### Prepare the test set



*   The test data can already be loaded in the section "loading the test dataset result frames of "Evaluation BM25"
*   However, if test data is not created there already, you can simply execute this block here


In [21]:
_, prep_question_dict, text_dict, id_title_dict, question_dict = load_all_dictionaries()

In [22]:
#load the testset
test_list = list()
test_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_5000.csv', sep='\t'))
test_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_15000.csv', sep='\t'))
test_list.append(pd.read_csv('./gdrive/MyDrive/Information_Retrieval_Project/bm25_outputs/bm25+_test_last.csv', sep='\t'))

df_test = pd.concat(test_list)

del df_test['Unnamed: 0']
df_test.reset_index(inplace=True)
del df_test['index']

#split the dataframes according to the question
df_test_list = list()
for question, df_question in df_test.groupby('question'):
  #del df_question['level_0']
  df_question.reset_index(inplace=True)
  del df_question['index']
  df_test_list.append(df_question)

add_labels(df_test_list, prep_question_dict)

test_df = pd.concat(df_test_list)

In [ ]:
display(test_df.head())

,question,doc,score,label
0,bb_0,25823,18.015410,1
1,bb_0,44424,17.880314,0
2,bb_0,1345,17.324262,0
3,bb_0,50244,16.947543,0
4,bb_0,20092,16.911382,0


In [ ]:
#write the test_df to the drive
test_df.to_csv('./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_test.csv', sep='\t')

### Prediction Methods for monoBERT

In [24]:
def predict_single_query(test_df, question_id, question_dict, id_title_dict, text_dict, model, tokenizer):
  """ scores the documents given a query
      ----------
      test_df: dataframe containing the bm25+ outputs
      question_id: id of question we want to predict for
  """
  temp_df = test_df[test_df['question'] == question_id]
  add_full_question_texts(temp_df, question_dict)
  add_full_doc_texts(temp_df, text_dict, id_title_dict)
  
  text_list = list(temp_df['full_doc'])
  question_list = list(temp_df['full_question'])
  
  encoding = tokenize_monoBERT(question_list, text_list, tokenizer)


  outputs = model(encoding['input_ids'], encoding['attention_mask'])
 
  temp_df['score'] = outputs.cpu().detach().numpy()
  
  return temp_df.sort_values(by=['score'], ascending=False)

def predict_multiple_queries(test_df, question_id_list, question_dict, id_title_dict, text_dict, model, tokenizer):
  """ scores the documents given a list of questions; returns a list of dataframes with ranked documents
      ----------
  """
  result_df_list = list()
  for i, question_id in enumerate(question_id_list):
    result_df_list.append(predict_single_query(test_df, question_id, question_dict, id_title_dict, text_dict, model, tokenizer)[0:10])
    print(i)

  return result_df_list


### Single query prediction workflow

In [27]:
#load the model from checkpoint and the tokenizer
_, tokenizer = get_monoBERT_and_tokenizer()
model = load_monoBERT('./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_lr_0.0001_DROPOUT_Adam_epoch_3.tar')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
#predict 
print(test_df.sample()['question'].values)
#interesting: qz_1854
output = predict_single_query(test_df, str(test_df.sample()['question'].values[0]), question_dict, id_title_dict, text_dict, model, tokenizer)


['qw_318']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
#output for the whole test set
result_df_list = predict_multiple_queries(test_df, test['question'].values, question_dict, id_title_dict, text_dict, model, tokenizer)

In [39]:
#create a sample for testing to reduce computation time
test_df_sample = test_df[7500:8500]

In [ ]:
display(test_df[7350:7400])

,question,doc,score,label
0,bb_1772,54626,38.652683,0
1,bb_1772,10401,34.972892,0
2,bb_1772,17688,31.368855,0
3,bb_1772,15327,30.926645,0
4,bb_1772,51834,30.469387,0
5,bb_1772,3477,29.725855,0
6,bb_1772,23251,29.566124,0
7,bb_1772,71215,29.386205,0
8,bb_1772,57302,29.314381,0
9,bb_1772,55929,29.289317,0


In [40]:
#predict for the chosen sample
all_results = predict_multiple_queries(test_df, test_df_sample.drop_duplicates('question')['question'], question_dict, id_title_dict, text_dict, model, tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

0
1
2
3


KeyboardInterrupt: ignored

In [30]:
#convert the result dataframe to a list to evaluate it later on
test_sample_list = list()
for question, df_question in test_df_sample.groupby('question'):
  df_question.reset_index(inplace=True)
  del df_question['index']
  test_sample_list.append(df_question)

In [96]:
#prediction quality for monoBERT

for i in range(1, 8):
  #MPK_bm = meanPrecisions(test_sample_list, prep_question_test, k=i, PType='P@K')
  MPK_monoBERT = meanPrecisions(all_results, prep_question_test, k=i, PType='P@K')
  print(" - monoBERT: ", MPK_monoBERT)
  #MAPK_bm = meanPrecisions(test_sample_list, prep_question_test, k=i, PType='AP@K')
  MAPK_monoBERT = meanPrecisions(all_results, prep_question_test, k=i, PType='AP@K')
  print(" - monoBERT: ", MAPK_monoBERT)
#MRPrecision_bm = meanPrecisions(test_sample_list, prep_question_test, k=5, PType='R-Precision')
MRPrecision_monoBERT = meanPrecisions(all_results, prep_question_test, k=5, PType='R-Precision')
print(" - monoBERT: ", MRPrecision_monoBERT)
#MARPrecision_bm = meanPrecisions(test_sample_list, prep_question_test, k=5, PType='AR-Precision')
MARPrecision_monoBERT = meanPrecisions(all_results, prep_question_test, k=5, PType='AR-Precision')
print(" - monoBERT: ", MARPrecision_monoBERT)

 - monoBERT:  0.7
 - monoBERT:  0.7
 - monoBERT:  0.5
 - monoBERT:  0.75
 - monoBERT:  0.4
 - monoBERT:  0.7333333333333333
 - monoBERT:  0.3375
 - monoBERT:  0.7194444444444444
 - monoBERT:  0.29000000000000004
 - monoBERT:  0.7144444444444444
 - monoBERT:  0.24166666666666664
 - monoBERT:  0.7144444444444444
 - monoBERT:  0.20714285714285707
 - monoBERT:  0.7144444444444444
 - monoBERT:  0.4916666666666666
 - monoBERT:  0.6916666666666667


# duoBERT

## Dataset/Loader and Model definition

In [28]:
# duoBERT and linear model
# Our duoBERT model with linear layers defined.
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        #torch.nn.init.zero_(m.bias)

class duoBERT(nn.Module):
    def __init__(self):
      super(duoBERT, self).__init__()
      self.bert = RobertaModel.from_pretrained('distilroberta-base')
      self.decision = nn.Sequential(
            nn.Linear(768, 128),
            nn.SiLU(),
            nn.Dropout(0.2),
            nn.Linear(128,1),
            nn.Sigmoid()
        )
      weights_init(self.decision)

    def forward(self, ids, mask):
      outputs = self.bert(ids, attention_mask=mask)

      sequence_output = outputs.last_hidden_state
      
      decision_output = self.decision(sequence_output[:,0,:].view(-1,768)) ## extract the 1st token's embeddings
      return decision_output

In [29]:

#Dataset for duoBERT is prepared
class duoBERT_dataset(Dataset):
  def __init__(self):
    #path = "./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/duoBERT_train.csv"
    #self.dataset = pd.read_csv(path, sep='\t')
    #self.dataset.reset_index(inplace=True)
    self.dataset = duo_train_df
  def __len__(self): 
    return len(self.dataset)
  
  def __getitem__(self, itemID):
    row = self.dataset.iloc[itemID]

    question = row['full_question']
    doc_x = row['full_doc_x']
    doc_y = row['full_doc_y']

    # Encoding the question
    question_encoding = tokenizer(question)
    question_inputids = question_encoding['input_ids']
    question_attentionmask = question_encoding['attention_mask']


    # Encoding the documents
    num_tokens_for_docx = math.floor((512-len(question_inputids))/2)+1
    num_tokens_for_docy = 512-len(question_inputids)-num_tokens_for_docx+2

    doc_x_encoding = tokenizer(doc_x, truncation=True, padding='max_length', max_length = num_tokens_for_docx)
    doc_x_inputids = doc_x_encoding['input_ids'][1:]
    doc_x_attentionmask = doc_x_encoding['attention_mask'][1:]
    doc_y_encoding = tokenizer(doc_y, truncation=True, padding='max_length', max_length = num_tokens_for_docy)
    doc_y_inputids = doc_y_encoding['input_ids'][1:]
    doc_y_attentionmask = doc_y_encoding['attention_mask'][1:]
    
    # We return the input ids for the whole input of 512 length
    all_ids = torch.LongTensor(question_inputids + doc_x_inputids + doc_y_inputids)
    all_attentionmask = torch.LongTensor(question_attentionmask + doc_x_attentionmask + doc_y_attentionmask)


    return all_ids, all_attentionmask, int(row['label'])

In [30]:
# Dataloader is prepared to iterate over batches 
# Get a dataloader for duoBERT
def get_dataloader_duoBERT():
  duoBERT_data = duoBERT_dataset()
  dataloader = DataLoader(dataset=duoBERT_data, batch_size=32, shuffle=True, num_workers=3)
  return dataloader

In [31]:
# Get a duoBERT model and tokenizer
def get_duoBERT_and_tokenizer():
  return duoBERT(), AutoTokenizer.from_pretrained('distilroberta-base', use_fast=True)

In [32]:
# Tokenizer for monoBERT
def tokenize_duoBERT(question, doc_x, doc_y, tokenizer):
# creates: [CLS] question [SEP] Doc_x [SEP] Doc_y [SEP], max 512 tokens
  
  # Encoding the question
  question_encoding = tokenizer(question)
  question_inputids = question_encoding['input_ids']
  question_attentionmask = question_encoding['attention_mask']
  
  # Encoding the documents
  num_tokens_for_docx = math.floor((512-len(question_inputids))/2)+1
  num_tokens_for_docy = 512-len(question_inputids)-num_tokens_for_docx+2

  doc_x_encoding = tokenizer(doc_x, truncation=True, padding='max_length', max_length = num_tokens_for_docx)
  doc_x_inputids = doc_x_encoding['input_ids'][1:]
  doc_x_attentionmask = doc_x_encoding['attention_mask'][1:]
  doc_y_encoding = tokenizer(doc_y, truncation=True, padding='max_length', max_length = num_tokens_for_docy)
  doc_y_inputids = doc_y_encoding['input_ids'][1:]
  doc_y_attentionmask = doc_y_encoding['attention_mask'][1:]
  
  # We return the input ids for the whole input of 512 length
  all_ids = torch.LongTensor([question_inputids + doc_x_inputids + doc_y_inputids])
  all_attentionmask = torch.LongTensor([question_attentionmask + doc_x_attentionmask + doc_y_attentionmask])
  all_ids
  all_attentionmask
  return all_ids, all_attentionmask

In [33]:
def load_duoBERT(path):
  model = duoBERT()
  model.load_state_dict(torch.load(path)["model_state_dict"])
  model.eval()
  return model

In [34]:
#doubert cnn based on the paper of Safaya et al. 2020 (self-implemented)
#link: https://www.aclweb.org/anthology/2020.semeval-1.271.pdf
#did not converge and needs a lot of training time 

'''class duoBERT_cnn(nn.Module):
    def __init__(self):
      super(duoBERT_cnn, self).__init__()
      self.bert = RobertaModel.from_pretrained('distilroberta-base')
      self.convolution1 = nn.Conv2d(6,3,(768,1),padding=True)
      self.convolution2 = nn.Conv2d(6,3,(768,2),padding=True)
      self.convolution3 = nn.Conv2d(6,3,(768,3), padding=True)
      self.convolution4 = nn.Conv2d(6,3,(768,4), padding=True)
      self.convolution5 = nn.Conv2d(6,3,(768,5),padding=True)
      self.maxPool = nn.MaxPool2d(kernel_size=3)
      self.silu = nn.SiLU()
      self.linear = nn.Linear(2556, 256)
      self.dropout = nn.Dropout(0.2)
      self.final = nn.Linear(256,1)
      #self.sigmoid = nn.Sigmoid()
      
      weights_init(self.convolution1)
      weights_init(self.convolution2)
      weights_init(self.convolution3)
      weights_init(self.convolution4)
      weights_init(self.convolution5)
      weights_init(self.linear)
      weights_init(self.final)

    def forward(self, ids, mask):
      outputs = self.bert(ids, attention_mask=mask, output_hidden_states = True)

      #print(len(outputs))  # 3

      hidden_states = outputs[2]
      #print(type(hidden_states))
      #print(len(hidden_states))  # 13

      emb_out = hidden_states[0]
      att_layers = hidden_states[1:]
      a = torch.stack([att_layers[1],att_layers[2],att_layers[3],att_layers[4], att_layers[5], emb_out])

      x = a.view(-1,6,768,512)
      #print("1:",x.shape)
      x1 = self.maxPool(self.silu(self.convolution1(x)))
      x2 = self.maxPool(self.silu(self.convolution2(x)))
      x3 = self.maxPool(self.silu(self.convolution3(x)))
      x4 = self.maxPool(self.silu(self.convolution4(x)))
      x5 = self.maxPool(self.silu(self.convolution5(x)))
     
      x1 = x1.view(-1, 513)
      x2 = x2.view(-1, 513)
      x3 = x3.view(-1, 510)
      x4 = x5.view(-1, 510)
      x5 = x5.view(-1, 510)

      x = torch.cat([x1,x2,x3,x4,x5],dim=1)

      x = self.silu(self.dropout((self.linear(x))))
      x = self.final(x)
      return x'''
#not used, as it did not converge properly

'class duoBERT_cnn(nn.Module):\n    def __init__(self):\n      super(duoBERT_cnn, self).__init__()\n      self.bert = RobertaModel.from_pretrained(\'distilroberta-base\')\n      self.convolution1 = nn.Conv2d(6,3,(768,1),padding=True)\n      self.convolution2 = nn.Conv2d(6,3,(768,2),padding=True)\n      self.convolution3 = nn.Conv2d(6,3,(768,3), padding=True)\n      self.convolution4 = nn.Conv2d(6,3,(768,4), padding=True)\n      self.convolution5 = nn.Conv2d(6,3,(768,5),padding=True)\n      self.maxPool = nn.MaxPool2d(kernel_size=3)\n      self.silu = nn.SiLU()\n      self.linear = nn.Linear(2556, 256)\n      self.dropout = nn.Dropout(0.2)\n      self.final = nn.Linear(256,1)\n      #self.sigmoid = nn.Sigmoid()\n      \n      weights_init(self.convolution1)\n      weights_init(self.convolution2)\n      weights_init(self.convolution3)\n      weights_init(self.convolution4)\n      weights_init(self.convolution5)\n      weights_init(self.linear)\n      weights_init(self.final)\n\n    def f

## Training preparation

In [ ]:
#load the training data
csv_path = "./gdrive/MyDrive/Information_Retrieval_Project/monoBERT/monoBERT_train.csv"
df = pd.read_csv(csv_path, sep='\t')

In [ ]:
display(df.head(10))

,Unnamed: 0,question,doc,label,full_question,full_doc
0,0,bb_10,27062,0,In the US TV comedy show Everybody Loves Raymo...,Becker is an American sitcom that ran from 199...
1,1,bb_10,72724,1,In the US TV comedy show Everybody Loves Raymo...,Everybody Loves Raymond is an American televis...
2,2,bb_1000,71300,0,An adaptation of which classic 1905 story by E...,The Railway Children is a children's book by E...
3,3,bb_1000,5640,0,An adaptation of which classic 1905 story by E...,John Barrymore (born John Sidney Blyth; Februa...
4,4,bb_1000,72730,1,An adaptation of which classic 1905 story by E...,Edith Nesbit (married name Edith Bland; 15 Aug...
5,5,bb_1000,11920,1,An adaptation of which classic 1905 story by E...,London Waterloo station ( ) is a central Lond...
6,6,bb_1011,37131,0,"Patricia Neal, winner of best actress Oscar fo...",Hud is a 1963 Western film directed by Martin ...
7,7,bb_1011,62787,1,"Patricia Neal, winner of best actress Oscar fo...","Patricia Neal (January 20, 1926 – August 8, 20..."
8,8,bb_1014,11682,0,Welsh rugby union flanker Andy Powell was bann...,Newport (; ) is a cathedral and university cit...
9,9,bb_1014,16945,1,Welsh rugby union flanker Andy Powell was bann...,"Andrew ""Andy"" Powell (born 23 August 1981) is ..."


In [ ]:
#filter for relevant and irrelevant documents
important_match = df[df["label"]==1]
unimportant_match = df[df["label"]==0]
del important_match['Unnamed: 0']
del unimportant_match['Unnamed: 0']

In [ ]:
display(important_match.head())
display(unimportant_match.head())

In [ ]:
#merge both dataframes
populated_df = important_match.merge(unimportant_match[["full_question","doc","full_doc"]],left_index=True,how='left', on=['full_question'])

In [ ]:
populated_df.reset_index(inplace=True)
populated_df = populated_df.rename(columns={'question': 'question_ID'})

In [ ]:
display(populated_df.head(10))

In [ ]:
#reverse the dataframe to create the opposite tuple with the opposite label
populated_reverse = pd.DataFrame(list(zip(list(populated_df['question_ID']), list(populated_df['doc_y']), [0]*len(populated_df), list(populated_df['full_question']), list(populated_df['full_doc_y']), list(populated_df['doc_x']), list(populated_df['full_doc_x']))),
               columns =['question_ID', 'doc_x', 'label', 'full_question', 'full_doc_x', 'doc_y', 'full_doc_y'])

In [ ]:
duo_train_df = pd.concat([populated_df, populated_reverse])
#del duo_train_df['level_0']
del duo_train_df['index']
duo_train_df.reset_index(inplace=True)
del duo_train_df['index']

In [ ]:
display(populated_df.head(10))
display(populated_reverse.head(10))

In [ ]:
display(duo_train_df)

In [ ]:
duo_train_df = duo_train_df[["question_ID","full_question","doc_x","full_doc_x","doc_y","full_doc_y","label"]]

In [ ]:
display(duo_train_df)

In [ ]:
duo_train_df.to_csv('./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/duoBERT_train.csv', sep='\t')

## Model initialization and training

In [ ]:
# Get a dataloader
loader = get_dataloader_duoBERT()

In [ ]:
# Criterion and Optimizer are defined for training
#criterion = nn.BCELoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)


In [ ]:
_, tokenizer = get_duoBERT_and_tokenizer()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = duoBERT()
model.to('cuda')

In [ ]:
# TRAINING
epochs = 100

for epoch in range(epochs):
  avg_loss = 0 
  print("EPOCH")
  for i, batch in enumerate(loader):
    input_ids = batch[0].to('cuda')
    attention_mask = batch[1].to('cuda')
    target = batch[2].to('cuda')
    optimizer.zero_grad()   
    #print(question)
    #print(encoding)
    #get token_ids and attention_mask from encoding
    outputs = model(input_ids, attention_mask)
    #outputs = F.log_softmax(outputs, dim=1)
    
    outputs = outputs.flatten()
    loss = criterion(outputs, target.to(torch.float32))
    avg_loss += loss
    if i % 30 == 10:
      print(i)
      print(outputs)
      print(target)
      print("Batch_loss: ", loss)
      print("Avg_loss: {}".format(avg_loss/i))
    loss.backward()
    optimizer.step()
  
    if  i == 5000:
      print("Save the weights")
      torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
      }, "./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/duoBERT_lr0001{}.tar".format(epoch))

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
tensor([0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1], device='cuda:0')
Batch_loss:  tensor(0.2101, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Avg_loss: 0.3639645278453827
2530
tensor([0.9895, 0.4383, 0.0074, 0.3619, 0.9908, 0.9945, 0.9919, 0.9933, 0.9813,
        0.7000, 0.1055, 0.9774, 0.9935, 0.9911, 0.4464, 0.0107, 0.0429, 0.5564,
        0.0292, 0.9955, 0.0133, 0.0100, 0.9866, 0.0134, 0.9163, 0.1343, 0.1184,
        0.9857, 0.0163, 0.4138, 0.9894, 0.4018], device='cuda:0',
       grad_fn=<ViewBackward>)
tensor([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 1, 0], device='cuda:0')
Batch_loss:  tensor(0.2084, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Avg_loss: 0.3620312213897705
2560
tensor([0.9274, 0.7946, 0.9927, 0.6809, 0.1927, 0.0165, 0.5826, 0.9939, 0.0077,
        0.9879, 0.0348

KeyboardInterrupt: ignored

## Prediction Making

to use the duobert prediction, first run the monoBERT model and predict "all_results" on the "test_df_sample"; this can then be loaded into the duobert inference functions

In [97]:
model = load_duoBERT('./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/duoBERT_new_lr00011_999.tar')
model.to('cuda')

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


duoBERT(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [63]:
from psutil import virtual_memory

def get_pairwise_scores(id_list, att_list, model):

  output = model(torch.stack(id_list).squeeze().to('cuda'), torch.stack(att_list).squeeze().to('cuda'))

  return output.sum()

def rank(question_id, df, model):
  """ sum up the scores of each document by comparing it with each other doc and passing it through the model
      ----------
  """
  score_dict = dict()
  temp_df = df[df['question'] == question_id]
  doc_ids = set(temp_df['doc'])
  question = temp_df.iloc[0]['question']
  for doc in temp_df['doc']:
    others = doc_ids - set([doc]) 
    sum = 0
    id_list = list()
    att_list = list()
    for doc_y in others:
      ids, attention_mask = tokenize_duoBERT(question, temp_df['full_doc'][temp_df['doc'] == doc].item(), temp_df['full_doc'][temp_df['doc'] == doc_y].item(), tokenizer)
      id_list.append(ids)
      att_list.append(attention_mask)
    sum += get_pairwise_scores(id_list[0:5], att_list[0:5], model)
    #sum += get_pairwise_scores(id_list[5:10], attention_mask[5:10], model)
    
    score_dict[doc] =sum.item()
  return score_dict

def rank_list(df_list, model):
  """ Takes a list of dataframes with ranked documents and reranks with the "rank" function
      ----------
  """
  result_list = list()
  for i, df in enumerate(df_list):
    df['new_score'] = df['doc'].map(rank(df.iloc[0]['question'], df[0:5], model))
    result_list.append(df.sort_values(['new_score'], ascending=False)[0:5])
  return result_list

def final_ranker(docs, pairwise_scores):
# docs -> List of the documents to be sorted
# pairwise_scores_rows -> Output dataset from duoBERT. It contains pairwise combinations of all documents, and an importance score per pairwise comparison

  # Empty scores dictionary to store the sum scores
  scores = {}
  for doc in docs:

    # Score per doc is initialized
    scores[doc] = 0

    for row in pairwise_scores:
        if row['doc_text_x'] == doc:
          # Sum scores calculated
          scores[doc] += row['score']

  # Order them by their score 
  scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

  # We define the final list to return, ranked from most relevant to least
  final_list = []
  for key, value in scores.items():
    final_list.append(key)

  
  return final_list


In [98]:
torch.cuda.empty_cache()

pairwise_scores = rank_list(all_results, model)

In [89]:
display(pairwise_scores[0])
display(pairwise_scores[1])
display(pairwise_scores[2])
display(pairwise_scores[3])
display(pairwise_scores[4])
display(pairwise_scores[5])
display(pairwise_scores[6])
display(pairwise_scores[7])
display(pairwise_scores[8])
display(pairwise_scores[9])


,question,doc,score,label,full_question,full_doc,new_score
3,bb_1966,41226,5.597218,1,The genocide of Tutsi people by Hutu militia i...,"The Tutsi (; ), or Abatutsi, are a population ...",3.842031
43,bb_1966,17510,5.098068,0,The genocide of Tutsi people by Hutu militia i...,Genocide (From Greek: Γενοκτονία) is the inten...,3.817302
14,bb_1966,30017,4.016120,0,The genocide of Tutsi people by Hutu militia i...,Genocide is the deliberate and systematic dest...,3.143354
4,bb_1966,38912,2.125228,1,The genocide of Tutsi people by Hutu militia i...,"The Hutu, also known as the Abahutu, are a pop...",2.837613
31,bb_1966,28671,0.826817,0,The genocide of Tutsi people by Hutu militia i...,Events\n\nJanuary\n\n* January 1\n** Cultivars...,2.743052


,question,doc,score,label,full_question,full_doc,new_score
32,bb_1972,47011,2.549249,0,"What on a bird has a calamus, rachis and barbs?","Barbed wire, also known as barb wire, less oft...",3.841907
12,bb_1972,60337,-3.442049,0,"What on a bird has a calamus, rachis and barbs?",Rattan (from the Malay rotan) is the name for ...,3.487921
7,bb_1972,46017,5.300384,1,"What on a bird has a calamus, rachis and barbs?",Rachis is a biological term for a main axis or...,3.422784
11,bb_1972,25719,-0.144353,0,"What on a bird has a calamus, rachis and barbs?",The emu (Dromaius novaehollandiae) is the seco...,2.735590
3,bb_1972,29719,-3.116462,0,"What on a bird has a calamus, rachis and barbs?",The Eurasian blue tit (Cyanistes caeruleus ) i...,1.329286


,question,doc,score,label,full_question,full_doc,new_score
9,bb_1979,25965,4.791040,1,Arthroscopy refers to surgery performed on wha...,Surgery (from the cheirourgikē (composed of χ...,3.746826
5,bb_1979,39622,0.546637,0,Arthroscopy refers to surgery performed on wha...,Minimally-invasive procedures (also known as m...,3.626589
29,bb_1979,31135,-2.774645,0,Arthroscopy refers to surgery performed on wha...,Organ transplantation is the moving of an orga...,2.926305
0,bb_1979,506,4.859808,1,Arthroscopy refers to surgery performed on wha...,Arthroscopy (also called arthroscopic surgery)...,2.787249
10,bb_1979,10836,-3.113057,0,Arthroscopy refers to surgery performed on wha...,Cardiovascular (heart) surgery is surgery on t...,2.753747


,question,doc,score,label,full_question,full_doc,new_score
46,bb_1988,53395,4.811922,0,"What is the Latin term for a reigning queen, o...",Augustus (;Classical Latin spelling and recons...,3.978975
38,bb_1988,3402,2.030160,0,"What is the Latin term for a reigning queen, o...",Mary I (18 February 1516 – 17 November 1558) w...,3.649093
13,bb_1988,62975,4.513467,0,"What is the Latin term for a reigning queen, o...",Elizabeth I (7 September 1533 – 24 March 1603)...,3.602562
33,bb_1988,35769,2.643804,0,"What is the Latin term for a reigning queen, o...",Henry VIII (28 June 1491 – 28 January 1547) wa...,3.562195
8,bb_1988,12295,2.626196,0,"What is the Latin term for a reigning queen, o...","The monarchy of the United Kingdom, commonly r...",3.241958


,question,doc,score,label,full_question,full_doc,new_score
0,bb_199,53893,-0.262293,1,"Angelos Epithemiou ex-burger-van proprietor, r...",Daniel Renton Skinner (born 25 January 1973) i...,3.891666
3,bb_199,55437,2.338067,0,"Angelos Epithemiou ex-burger-van proprietor, r...","Friends is an American television sitcom, crea...",3.645027
44,bb_199,28226,0.460003,0,"Angelos Epithemiou ex-burger-van proprietor, r...",Popular is an American teenage comedy-drama on...,3.475925
5,bb_199,38741,-0.116815,0,"Angelos Epithemiou ex-burger-van proprietor, r...",The British Comedy Awards are an annual awards...,2.493372
22,bb_199,29334,2.610252,0,"Angelos Epithemiou ex-burger-van proprietor, r...",3–2–1 was a popular (and successful) British g...,2.306855


,question,doc,score,label,full_question,full_doc,new_score
35,bb_1994,29776,-1.019057,0,"Dame Evelyn Glennie, who is profoundly deaf, i...",Henry Graham Greene (2 October 1904 – 3 April...,3.829299
40,bb_1994,30930,-0.114778,0,"Dame Evelyn Glennie, who is profoundly deaf, i...","Hearing loss, also known as hearing impairment...",3.759499
21,bb_1994,4605,-0.972717,0,"Dame Evelyn Glennie, who is profoundly deaf, i...","Dame Judith Olivia ""Judi"" Dench, (born 9 Dece...",2.687468
0,bb_1994,46061,4.420423,1,"Dame Evelyn Glennie, who is profoundly deaf, i...","Dame Evelyn Elizabeth Ann Glennie, DBE (born 1...",2.548301
24,bb_1994,57914,-0.056605,0,"Dame Evelyn Glennie, who is profoundly deaf, i...",British literature is literature in the Englis...,2.473960


,question,doc,score,label,full_question,full_doc,new_score
5,bb_2001,2983,6.399828,1,"The mineral, diamond, is naturally what crysta...",A mineral is a naturally occurring chemical co...,3.964266
0,bb_2001,28224,6.558377,1,"The mineral, diamond, is naturally what crysta...",Diamond ( or) is a metastable allotrope of car...,3.801628
41,bb_2001,28525,5.747653,0,"The mineral, diamond, is naturally what crysta...","In crystallography, crystal structure is a des...",3.118227
48,bb_2001,43885,5.863828,1,"The mineral, diamond, is naturally what crysta...",A crystal or crystalline solid is a solid mate...,1.650063
6,bb_2001,43196,5.561342,0,"The mineral, diamond, is naturally what crysta...",A diamond (from the ancient Greek ἀδάμας – adá...,1.622483


,question,doc,score,label,full_question,full_doc,new_score
15,bb_2019,24318,5.141289,0,"The FischerSaller scale, used in anthropology...",Medicine (British English; American English) i...,3.812071
30,bb_2019,64418,0.982765,0,"The FischerSaller scale, used in anthropology...",Violence is defined by the World Health Organi...,3.744488
0,bb_2019,64674,5.051980,1,"The FischerSaller scale, used in anthropology...","The Fischer-Saller Scale, named after Eugen Fi...",3.622931
8,bb_2019,65141,-0.182845,0,"The FischerSaller scale, used in anthropology...","Race, as a social construct, is a group of peo...",3.201683
9,bb_2019,62797,3.464075,0,"The FischerSaller scale, used in anthropology...",Eye color or eye colour is a polygenic phenoty...,2.857603


,question,doc,score,label,full_question,full_doc,new_score
37,bb_2030,51484,-0.027904,0,Name the former News of the World royal corres...,The original News Corporation or News Corp. wa...,3.948065
1,bb_2030,67304,4.811405,1,Name the former News of the World royal corres...,The News of the World was a national red top n...,3.627759
3,bb_2030,33082,2.367293,0,Name the former News of the World royal corres...,Watergate was a major political scandal that o...,2.582930
0,bb_2030,54141,3.473249,1,Name the former News of the World royal corres...,The News International phone-hacking scandal i...,2.295739
35,bb_2030,11007,4.556123,0,Name the former News of the World royal corres...,Phone hacking is the practice of intercepting ...,1.950437


,question,doc,score,label,full_question,full_doc,new_score
4,bb_2036,73299,4.433249,1,"Under the ABO blood group system, what blood t...",A blood donation occurs when a person voluntar...,3.902734
0,bb_2036,66944,5.443734,1,"Under the ABO blood group system, what blood t...",A blood type (also called a blood group) is a ...,2.745464
17,bb_2036,31704,1.660743,0,"Under the ABO blood group system, what blood t...","Red blood cells (RBCs), also called erythrocyt...",2.625123
1,bb_2036,46250,5.349922,1,"Under the ABO blood group system, what blood t...",The ABO blood group system is the most importa...,2.238465
2,bb_2036,32084,4.112684,1,"Under the ABO blood group system, what blood t...",Blood transfusion is generally the process of ...,1.395283


In [33]:
#create dataframe with answers to the questions
questionID = list()
answers = list()
for ele in df['Data']:
  questionID.append(ele['QuestionId'])
  answers.append(ele['Answer']['Value'])



  data = {'questionID': questionID,
          'answer': answers,
  }
answer_df = pd.DataFrame(data, columns=data.keys())

In [ ]:
display(answer_df)

,questionID,answer
0,tc_3,York
1,tc_8,Portugal
2,tc_9,Chicago
3,tc_10,Chicago Bears
4,tc_11,Norway
...,...,...
61883,qg_4639,"Russia, China, Kyrgyzstan, Uzbekistan, and Tur..."
61884,qg_4642,Barbie
61885,qg_4643,Theodore Roosevelt
61886,qg_4646,Bugs Bunny


In [81]:
#add answers to the result dataframe to pass it to the question answering model
for df in pairwise_scores:
  df['answer'] = df['question'].apply(lambda x: answer_df['answer'][answer_df['questionID'] == df.iloc[0]['question']])

In [ ]:
pairwise_scores[0].to_csv('./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/test1.csv')

pairwise_scores[7].to_csv('./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/finaltest2.csv')

In [37]:
for i,df in enumerate(test_sample_list):
  df['answer'] = df['question'].apply(lambda x: answer_df['answer'][answer_df['questionID'] == df.iloc[0]['question']])
  add_full_doc_texts(df, question_dict, id_title_dict)
  add_full_question_texts(df, question_dict)
  df[0:5].to_csv('./gdrive/MyDrive/Information_Retrieval_Project/duoBERT/final{}.csv'.format(i))


In [38]:
display(test_sample_list[0])
display(test_sample_list[1])

,question,doc,score,label,answer,full_doc,full_question
0,bb_1905,26962,19.597762,0,BBC,"Messiah (HWV 56), the English-language oratori...",Nation Shall Speak Peace Unto Nation.
1,bb_1905,27523,18.889829,0,BBC,"In the Abrahamic religions, Gabriel ( Gavri'el...",Nation Shall Speak Peace Unto Nation.
2,bb_1905,62572,17.887833,0,BBC,"Sola fide (Latin: by faith alone), also known ...",Nation Shall Speak Peace Unto Nation.
3,bb_1905,66731,17.275548,0,BBC,The Lamanites are one of the people described...,Nation Shall Speak Peace Unto Nation.
4,bb_1905,29155,17.145931,0,BBC,"The Annunciation (from the Vulgate Latin '), ...",Nation Shall Speak Peace Unto Nation.
5,bb_1905,40194,16.286592,0,BBC,"In the Hebrew Bible and the Quran, Aaron אַהֲר...",Nation Shall Speak Peace Unto Nation.
6,bb_1905,72235,16.109563,1,BBC,The coat of arms of the BBC was adopted in Mar...,Nation Shall Speak Peace Unto Nation.
7,bb_1905,4651,16.061345,0,BBC,The Beatitudes are eight blessings recounted i...,Nation Shall Speak Peace Unto Nation.
8,bb_1905,29611,15.356243,0,BBC,"The Ten Commandments, also known as the Decalo...",Nation Shall Speak Peace Unto Nation.
9,bb_1905,55879,15.248792,0,BBC,"A doxology (Greek: , from , doxa, ""glory"" and ...",Nation Shall Speak Peace Unto Nation.


,question,doc,score,label,answer,full_doc,full_question
0,bb_1909,66058,31.622703,1,The Royal Society,"Nullius in verba (Latin for ""on the word of no...",Nullius in verba. (On the word of no one.)
1,bb_1909,13420,23.467662,0,The Royal Society,"The President, Council, and Fellows of the Roy...",Nullius in verba. (On the word of no one.)
2,bb_1909,39891,13.118247,0,The Royal Society,Eddie Koiki Mabo (c. 29 June 1936 – 21 January...,Nullius in verba. (On the word of no one.)
3,bb_1909,54614,12.655831,0,The Royal Society,Palm Sunday is a Christian moveable feast that...,Nullius in verba. (On the word of no one.)
4,bb_1909,34152,12.331806,0,The Royal Society,A figure of speech or rhetorical figure is fi...,Nullius in verba. (On the word of no one.)
5,bb_1909,57746,12.061370,0,The Royal Society,"In Greek mythology, the Pierian Spring of Mace...",Nullius in verba. (On the word of no one.)
6,bb_1909,53305,11.708740,0,The Royal Society,"An onomatopoeia (, or chiefly NZ; from the G...",Nullius in verba. (On the word of no one.)
7,bb_1909,57677,11.704328,0,The Royal Society,The Constitutional history of Australia began ...,Nullius in verba. (On the word of no one.)
8,bb_1909,38166,11.644904,0,The Royal Society,"A sign is an object, quality, event, or entity...",Nullius in verba. (On the word of no one.)
9,bb_1909,46715,11.437795,0,The Royal Society,Colonization (or colonisation) is an ongoing p...,Nullius in verba. (On the word of no one.)


In [82]:
display(pairwise_scores[7])

,question,doc,score,label,full_question,full_doc,new_score,answer
15,bb_2019,24318,5.141289,0,"The FischerSaller scale, used in anthropology...",Medicine (British English; American English) i...,3.812071,Hair
30,bb_2019,64418,0.982765,0,"The FischerSaller scale, used in anthropology...",Violence is defined by the World Health Organi...,3.744488,Hair
0,bb_2019,64674,5.051980,1,"The FischerSaller scale, used in anthropology...","The Fischer-Saller Scale, named after Eugen Fi...",3.622931,Hair
8,bb_2019,65141,-0.182845,0,"The FischerSaller scale, used in anthropology...","Race, as a social construct, is a group of peo...",3.201683,Hair
9,bb_2019,62797,3.464075,0,"The FischerSaller scale, used in anthropology...",Eye color or eye colour is a polygenic phenoty...,2.857603,Hair


In [100]:
#evaluate duoBERT
for i in range(1, 6):
  MPK_duoBERT = meanPrecisions(pairwise_scores, prep_question_test, k=i, PType='P@K')
  print("p@k - monoBERT: ", MPK_duoBERT)
  MAPK_duoBERT = meanPrecisions(pairwise_scores, prep_question_test, k=i, PType='AP@K')
  print("ap@k - monoBERT: ", MAPK_duoBERT)
MRPrecision_duoBERT = meanPrecisions(pairwise_scores, prep_question_test, k=5, PType='R-Precision')
print("R-precision - monoBERT: ", MRPrecision_duoBERT)
MARPrecision_duoBERT = meanPrecisions(pairwise_scores, prep_question_test, k=5, PType='AR-Precision')
print("AR-precision - monoBERT: ", MARPrecision_duoBERT)

p@k - monoBERT:  0.4
ap@k - monoBERT:  0.4
p@k - monoBERT:  0.3
ap@k - monoBERT:  0.45
p@k - monoBERT:  0.23333333333333334
ap@k - monoBERT:  0.4833333333333333
p@k - monoBERT:  0.2625
ap@k - monoBERT:  0.4499999999999999
p@k - monoBERT:  0.29000000000000004
ap@k - monoBERT:  0.4685416666666667
R-precision - monoBERT:  0.31250000000000006
AR-precision - monoBERT:  0.4624999999999999
